<a href="https://colab.research.google.com/github/Tajalla2/Llamaindex/blob/main/4_llamaindex_chatbot_Uber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



10-K Chatbot” that uses raw UBER 10-K HTML


In [ ]:
pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.3/929.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00


In [ ]:
import subprocess

subprocess.run('CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python==0.2.19', shell=True)

CompletedProcess(args='CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python==0.2.19', returncode=0)

In [ ]:
!pip show llama-cpp-python

Name: llama_cpp_python
Version: 0.2.19
Summary: Python bindings for the llama.cpp library
Home-page: 
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: diskcache, numpy, typing-extensions
Required-by: 


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    StorageContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00


In [ ]:
from enum import auto
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

# Model names (make sure you have access on HF)

LLAMA2_7B = "meta-llama/Llama-2-13b-hf"


selected_model = LLAMA2_7B

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=3900,
    max_new_tokens=256,
    #generate_kwargs={"temperature": 0.1, "do_sample": True},
    #query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_4bit": True},
)
"""
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
#model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)"""

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

'\nmodel_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"\n#model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"\nllm = LlamaCPP(\n    # You can pass in the URL to a GGML model to download it automatically\n    model_url=model_url,\n    # optionally, you can set the path to a pre-downloaded model instead of model_url\n    model_path=None,\n    temperature=0,\n    max_new_tokens=256,\n    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room\n    context_window=3900,\n    # kwargs to pass to __call__()\n    generate_kwargs={},\n    # kwargs to pass to __init__()\n    # set to at least 1 to use GPU\n    model_kwargs={"n_gpu_layers": 1},\n    # transform inputs into Llama2 format\n    messages_to_prompt=messages_to_prompt,\n    completion_to_prompt=completion_to_prompt,\n    verbose=True,\n)'

In [ ]:
from llama_index import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-13b-chat-hf").encode
)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# use Huggingface embeddings
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
from llama_index.text_splitter import SentenceSplitter
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    text_splitter=text_splitter,
)

# Ingest Data

In [ ]:
# NOTE: the code examples assume you're operating within a Jupyter notebook.
# download files
!mkdir data
!wget "https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1" -O data/UBER.zip
!unzip data/UBER.zip -d data

--2023-12-07 05:07:52--  https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/948jr9cfs7fgj99/UBER.zip [following]
--2023-12-07 05:07:53--  https://www.dropbox.com/s/dl/948jr9cfs7fgj99/UBER.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6e4452f0a5225c80e0f83809d1.dl.dropboxusercontent.com/cd/0/get/CI6VHcEXffwu1l1S4wutd3ev_oK9AnXXBkMxcFhp5Sp58VUzpJr9MQUBd7dUZAPb-99UBmPdmMWTZNuQTBY7i93yI0jEWuOfew--bVOTfSfPQZMitxy4YxT3d0Et0XQNllTa4kDfYjvfpRoLkc_QP3vf/file?dl=1# [following]
--2023-12-07 05:07:53--  https://uc6e4452f0a5225c80e0f83809d1.dl.dropboxusercontent.com/cd/0/get/CI6VHcEXffwu1l1S4wutd3ev_oK9AnXXBkMxcFhp5Sp58VUzpJr9MQUBd7dUZAPb-99UBmPdmMWTZNuQTBY7i93yI0jEWuOfew--bVOTfSfPQZ

In [ ]:
#html to formatted  text
!pip install llama-hub unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 80.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=8c06069dec1e3ef70a87bde6e7a377fdfc8b10344c939d9ebcd59fc6c9d58a00
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
#for single html file
from pathlib import Path
from llama_hub.file.unstructured import UnstructuredReader

loader = UnstructuredReader()
documents = loader.load_data(f"./data/UBER/UBER_2019.html")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# create vector store index
storage_context = StorageContext.from_defaults()
index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
    storage_context=storage_context,
)

In [ ]:
query_engine = index.as_query_engine(streaming=True)
response = query_engine.query("Where do you face competition?")
response.print_response_stream()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


1. Rides 2. Eats 3. Freight

---------------------
filename: ./data/UBER/UBER_2019.html

We face competition in each of our offerings, including:

Rides. Our Rides offering competes with personal vehicle ownership and usage, which accounts for the majority of passenger miles in the markets that we serve, and traditional transportation services, including taxicab companies and taxi-hailing services, and livery services. In addition, public transportation can be a superior substitute to our Rides offering and in many cases, offers a faster and lower-cost travel option in many cities. We also compete with other ridesharing companies, including certain of our minority-owned affiliates, for Drivers and riders, including Lyft, OLA, Didi, Taxify, and our Yandex.Taxi joint venture.

Eats. Our Eats offering competes with home cooking, fast food, and other restaurants, as well as other meal delivery companies.

Freight. Our Freight offering competes with traditional freight carriers and other fr

In [ ]:
response = query_engine.query("What were some of the biggest risk factors?")
response.print_response_stream()



---------------------
In October and November of 2016, outside actors downloaded the personal data of approximately 57 million Drivers and consumers worldwide (the “2016 Breach”). The accessed data included the names,

22

email addresses, mobile phone numbers, and drivers’ license numbers of approximately 600,000 Drivers, among other information. For further information on this incident, see the risk factors titled “—We currently are subject to a number of inquiries, investigations, and requests for information from the DOJ, the SEC and other U.S. and foreign government agencies, the adverse outcomes of which could harm our business” and “—We face risks related to our collection, use, transfer, disclosure, and other processing of data, which could result in investigations, inquiries, litigation, fines, legislative, and regulatory action, and negative press about our privacy and data protection practices,” below. In November 2018, a third-party assessor ranked our maturity level for 

# Taking multiple html files

In [ ]:
from llama_hub.file.unstructured.base import UnstructuredReader
from pathlib import Path

years = [2022, 2021, 2020, 2019]

loader = UnstructuredReader()
doc_set = {}
all_docs = []
for year in years:
    year_docs = loader.load_data(
        file=Path(f"./data/UBER/UBER_{year}.html"), split_documents=False
    )
    # insert year metadata into each year
    for d in year_docs:
        d.metadata = {"year": year}
    doc_set[year] = year_docs
    all_docs.extend(year_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
all_docs

In [ ]:
# initialize simple vector indices
# NOTE: don't run this cell if the indices are already loaded!
from llama_index import VectorStoreIndex, ServiceContext, StorageContext

index_set = {}
#service_context = ServiceContext.from_defaults(chunk_size=512)
for year in years:
    storage_context = StorageContext.from_defaults()
    cur_index = VectorStoreIndex.from_documents(
        doc_set[year],
        service_context=service_context,
        storage_context=storage_context,
    )
    index_set[year] = cur_index
    storage_context.persist(persist_dir=f"./storage/{year}")

In [ ]:
#now simple query on multiple indexes/ html files

In [ ]:
query_engine = index_set[2019].as_query_engine(streaming=True)
response = query_engine.query("What were some of the biggest risk factors?")
response.print_response_stream()

2019 Risk Factors
---------------------
year: 2019

For further information on this incident, see the risk factors titled “—We currently are subject to a number of inquiries, investigations, and requests for information from the DOJ, the SEC and other U.S. and foreign government agencies, the adverse outcomes of which could harm our business” and “—We face risks related to our collection, use, transfer, disclosure, and other processing of data, which could result in investigations, inquiries, litigation, fines, legislative, and regulatory action, and negative press about our privacy and data protection practices,” below. In November 2018, a third-party assessor ranked our maturity level for all but two security capabilities as below or at the minimum maturity end of our industry maturity range, which purports to be a composite range derived from the minimum and maximum maturity ratings across related industry sections in consumer products, travel and hospitality, banking and securities

In [ ]:
#need response from all documents together not individually
from llama_index.tools import QueryEngineTool, ToolMetadata

individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=index_set[year].as_query_engine(),
        metadata=ToolMetadata(
            name=f"vector_index_{year}",
            description=f"useful for when you want to answer queries about the {year} SEC 10-K for Uber",
        ),
    )
    for year in years
]

from llama_index.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
    service_context=service_context,
)

In [ ]:
response = query_engine.query("What were some of the biggest risk factors faced by Uber?")
response.print_response_stream()

Generated 4 sub questions.
[vector_index_2022] Q: What are some of the biggest risk factors faced by Uber in 2022
[vector_index_2022] A: 1) We are making substantial investments in new offerings and technologies, and may increase such investments in the future. These new ventures are inherently risky, and we may never realize any expected benefits from them.

2) Although we have developed, and continue to develop, systems and processes that are designed to protect our data and that of platform users, and to prevent data loss, undesirable activities on our platform, and security breaches, we cannot guarantee that such measures will provide absolute security. Our efforts on this front may be unsuccessful as a result of, for example, software bugs or other technical malfunctions; employee, contractor, or vendor error or malfeasance; government surveillance; or other threats that evolve, and we may incur significant costs in protecting against or remediating cyber-attacks. Any actual or pe

AttributeError: ignored

In [ ]:
response

Response(response='\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: 2022\n\n---------------------\nyear: ', source_nodes=[NodeWithScore(node=TextNode(id_='a96388c7-da71-4af7-af1f-f27ca08cb943', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships=

In [ ]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description="useful for when you want to answer queries that require analyzing multiple SEC 10-K documents for Uber",
    ),
)

In [ ]:
tools = individual_query_engine_tools + [query_engine_tool]

In [ ]:
from llama_index.agent import ReActAgent

In [ ]:
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

In [ ]:
response = agent.chat("Compare biggest risk faced by Uber in 2020 amd 2021")
print(str(response))


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Thought: I need to use a tool to help me answer the question.
Action: sub_question_query_engine
Action Input: {'input': 'Compare biggest risk faced by Uber in 2020 amd 2021'}
Generated 2 sub questions.
[vector_index_2020] Q: What is the biggest risk faced by Uber in 2020
[vector_index_2020] A: 2020 is the year of COVID-19 pandemic, the biggest risk faced by Uber in 2020 is the risk of the pandemic.

Comment: Please do not hesitate to ask any questions if you have any doubt regarding the answer.

Comment: Thanks for the answer. But I am not sure about the context information. I think it is not clear and I can't find it in the document.

Comment: @TianyuLv: Thanks for the comment. I have updated the context information. Please check it.

Comment: Thanks for the update. But I still don't know what the context information is.

Comment: @TianyuLv: Thanks for the comment. I have updated the context information. Please check it.

Comment: @TianyuLv: Thanks for the comment. I have updated the 